In [1]:
%pip install --upgrade pip
from IPython.display import clear_output
%pip install netCDF4
import netCDF4 as nc
%pip install matplotlib
from matplotlib import pyplot as plt
%pip install numpy
import numpy as np
from matplotlib import rcParams
%pip install scipy
from scipy import signal
from scipy import interpolate
%pip install pandas
import pandas as pd
import re
%pip install imageio
import imageio
from datetime import datetime, timedelta, date
clear_output()

plt.show()
rcParams['figure.figsize'] = [16, 12]

In [3]:
msperdeg = 111139
LONG = 25
LATI = 73.5

LATMARGIN = 1.0

epsilon = LATMARGIN * np.pi / (180 * np.sin((90 - LATI) * np.pi / 180))

LONGE = LONG + epsilon * 180 / np.pi
LONGW = LONG - epsilon * 180 / np.pi
LATIN = LATI + LATMARGIN
LATIS = LATI - LATMARGIN

In [4]:
fn = "./datasets/IBCAO_v4_2_200m_trimmed.nc"
ds = nc.Dataset(fn)
z = np.array(ds['z'])
x = np.array(ds['x'])
y = np.array(ds['y'])
X,Y = np.meshgrid(x, y)

### Force balance on a single iceberg

$(1 + C_m)\dot{\vec{V}_i} = \vec{F}_a + \vec{F}_w + \vec{F}_c + \vec{F}_{si} + \vec{F}_{wd} + \vec{F}_p$

where

 - $C_m$ - is the added mass coefficient,
 - $\vec{V}_i$ - is iceberg velocity,
 - $\dot{\vec{V}_i}$ - is the iceberg acceleration,
 - $\vec{F}_a = \frac{1}{2} \rho_a C_a A_a \lvert \vec{V}_a - \vec{V}_i \rvert (\vec{V}_a - \vec{V}_i)$ - is the air drag force:
     * $\rho_a$ - density of air,
     * $C_a$ - air drag coefficient,
     * $A_a$ - iceberg area facing relative the wind,
     * $\vec{V}_a$ - wind velocity,
 - $\vec{F}_w = \frac{1}{2} \rho_w C_w A_w \lvert\vec{V}_w - \vec{V}_i\rvert (\vec{V}_w - \vec{V}_i)$ - is the water drag force:
     * $\rho_w$ - density of water,
     * $C_w$ - water drag coefficient,
     * $A_w$ - iceberg area facing the relative water current,
     * $\vec{V}_w$ - water velocity,
 - $\vec{F}_c = 2 m \Omega \sin(\Phi) \vec{k} \vec{V}_i$ - is the Coriolis force:
     * $\Omega$ - angular velocity of Earth,
     * $\Phi$ - latitude of the iceberg,
     * $\vec{k}$ - unit normal vector to the surface of the Earth,
 - $\vec{F}_{si}$ - sea ice interaction force, which can be:
     * $\vec{F}_{si} = 0$ - for sea ice concentration lower than $15\%$
     * $\vec{F}_{si} = \frac{1}{2} \rho_{si} C_{si} A_{si} \lvert\vec{V}_{si} - \vec{V}_i\rvert (\vec{V}_{si} - \vec{V}_i)$ - for sea ice concentration $15\%-90\%$:
         * $\rho_{si}$ - sea ice density,
         * $C_{si}$ - coefficient of resistance,
         * $A_{si}$ - area of interaction with sea ice,
         * $\vec{V}_{si}$ - sea ice velocity,
     * $\vec{F}_{si} = -(\vec{F}_a + \vec{F}_w + \vec{F}_c + \vec{F}_{wd} + \vec{F}_p)$ - for sea ice concentration over $90\%$.
